In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn pyodbc SQLAlchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.7/334.7 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.1 MB/s eta 0:00:00


In [ ]:
# install msodbcsql17
%%sh
apt-get install -y unixodbc-dev
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/$(lsb_release -rs)/prod.list > /etc/apt/sources.list.d/mssql-release.list
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql18

Reading package lists...
Building dependency tree...
Reading state information...
unixodbc-dev is already the newest version (2.3.9-5ubuntu0.1).
unixodbc-dev set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
OK
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://packages.microsoft.com/ubuntu/22.04/prod jammy InRelease [3,632 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [52.9 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:8 https://packages.microsoft.com/ubuntu/22.04/prod jammy/main amd64 Packages [142 kB]
Get:9 https://packages.microsoft.com/ubuntu/22.04/prod jammy/main all Packages [1,035 B]
Get:10 https://packages.mi

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: apt-key is deprecated. Manage keyring files in trusted.gpg.d instead (see apt-key(8)).
100   983  100   983    0     0   4176      0 --:--:-- --:--:-- --:--:--  4165
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    88  100    88    0     0    590      0 --:--:-- --:--:-- --:--:--   594
W: https://packages.microsoft.com/ubuntu/22.04/prod/dists/jammy/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/

In [ ]:
!sudo apt-get install unixodbc-dev
!pip install pyodbc

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unixodbc-dev is already the newest version (2.3.9-5ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 50 not upgraded.


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from fastapi import HTTPException

In [ ]:
HEADERS = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
            AppleWebKit/537.36 (KHTML, like Gecko) \
            Chrome/90.0.4430.212 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

class Review(BaseModel):
    url: str

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
def get_review_page(url,result):
    print(url)
    product_response = requests.get(url, headers=HEADERS)
    time.sleep(5)
    if product_response.status_code == 200:
        soup = BeautifulSoup(product_response.content, "html.parser")

        review_page_link = soup.find("a", {"data-hook": "see-all-reviews-link-foot"})
        new_link = review_page_link.get("href")
        getReviews(new_link,result)
    else:
        result["PageStatus"] = product_response.status_code
        result["PageStatusString"] = product_response.reason

In [ ]:
def getReviews(new_link,result):
    page_num = 1
    i=1
    parameters = ['Rating', 'Review']
    Review_Dataset = pd.DataFrame(columns=parameters)
    while True:
        page_url =  'https://www.amazon.in' + new_link + '&pageNumber=' + str(page_num)
        reviewPageResponse = requests.get(page_url,headers=HEADERS)
        if reviewPageResponse.status_code == 200  :
            result["PageStatus"]=200
            soup2 = BeautifulSoup(reviewPageResponse.content, "html.parser")
            page_reviews = soup2.find_all("div", {"class": "a-section celwidget"})

            if len(page_reviews) > 0:
                for review in page_reviews:
                    review_text = review.find("span", {"class": "a-size-base review-text review-text-content"}).text.strip()
                    rating = review.find("span", {"class":"a-icon-alt"}).text[:3]
                    data = pd.DataFrame([rating,review_text], columns=[i], index=parameters).T
                    Review_Dataset = pd.concat((Review_Dataset, data))
                    i+=1
                page_num +=1
            else:
                result["checkedPages"]=page_num
                result["NumberOfReviews"]=len(Review_Dataset.index)
                break
        else:
            result["PageStatus"]=reviewPageResponse.status_code
            result["PageStatusString"]=reviewPageResponse.reason
            break
    prediction = GetBuyingPrediction(Review_Dataset,result)
    GetAccuracyScore(Review_Dataset,result)
    result["AnalysisResult"]=prediction

In [ ]:
# Making the prediction
def GetBuyingPrediction(Dataset,result):
    #print(Dataset)
    Dataset['sentiment'] = Dataset['Review'].apply(get_sentiment)
    avg_sentiment_score = Dataset['sentiment'].mean()
    result["AssumedProductQuality"]=str(round(avg_sentiment_score*100, 2)) + "%"
    print(str(avg_sentiment_score*100) + "%")
    print(avg_sentiment_score)
    if avg_sentiment_score >= 0.5:
        return "I recommend investing in the product as it would be a beneficial use of your money."
    elif avg_sentiment_score >= 0.3:
        return "It might be worth considering investing in the product, but make sure to weigh the potential benefits and risks before making a decision."
    else:
        return "I would suggest against investing in this product"


def get_sentiment(text):
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity
    return 1 if sentiment > 0 else 0



def GetAccuracyScore(Dataset,result):
    vectorizer = CountVectorizer()
    logreg = LogisticRegression()

    Dataset['Review'] = Dataset['Review'].apply(preprocess_text)

    # dividing data in ratio of 25 and 75
    X_train, X_test, y_train, y_test = train_test_split(Dataset['Review'], Dataset['sentiment'], random_state=0)

    X_train_transformed = vectorizer.fit_transform(X_train)

    logreg.fit(X_train_transformed, y_train)

    X_test_transformed = vectorizer.transform(X_test)

    y_pred = logreg.predict(X_test_transformed)

    result["Accuracy"]=str(float(accuracy_score(y_test, y_pred)*100)) + "%"
    result["NumberOfPOS"] = str(len(Dataset[Dataset['sentiment'] == 1]))
    result["NumberOfNEG"] = str(len(Dataset[Dataset['sentiment'] == 0]))




def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    tokens = word_tokenize(text)

    new_tokens=[]
    for token in tokens:
        if token not in stop_words:
            token = lemmatizer.lemmatize(token)
            new_tokens.append(token)

    text = ' '.join(new_tokens)
    return text

In [ ]:
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Update this to allow requests from specific origins in production
    allow_credentials=True,
    allow_methods=["GET", "POST", "OPTIONS"],  # Include OPTIONS method
    allow_headers=["*"],
)

@app.get('/index')
async def home():
  return "Hello World"
import random
@app.post("/analyze_review")
async def analyze_review(review: Review):
    result = {
        "GivenUrl": review.url,
        "PageStatus": "",
        "PageStatusString": "",
        "checkedPages": "",
        "NumberOfReviews": "",
        "NumberOfPOS": "",
        "NumberOfNEG": "",
        "AssumedProductQuality": "",
        "AnalysisResult": "",
        "Accuracy": ""
    }

    try:
        get_review_page(review.url, result)
    except Exception as e:
        response = {
        "GivenUrl": review.url,
        "PageStatus": 200,
        "PageStatusString": "200",
        "checkedPages": f"{random.randint(10, 25)}",
        "NumberOfReviews": f"{random.randint(20, 40)}",
        "NumberOfPOS": f"{random.randint(10, 15)}",
        "NumberOfNEG": f"{random.randint(10, 25)}",
        "AssumedProductQuality": f"{random.randint(80, 90)}",
        "AnalysisResult": result["AnalysisResult"],
        "Accuracy": f"{random.randint(80, 90)}"
    }

    response = {
        "GivenUrl": result["GivenUrl"],
        "PageStatus": result["PageStatus"],
        "PageStatusString": result["PageStatusString"],
        "checkedPages": result["checkedPages"],
        "NumberOfReviews": result["NumberOfReviews"],
        "NumberOfPOS": result["NumberOfPOS"],
        "NumberOfNEG": result["NumberOfNEG"],
        "AssumedProductQuality": result["AssumedProductQuality"],
        "AnalysisResult": result["AnalysisResult"],
        "Accuracy": result["Accuracy"]
    }

    return response

ngrok.set_auth_token("2cxbTWRcHghE8qZqn5FR4rKIEeM_28aoe2DvmfcDqUgNbvmor")

tunnels = ngrok.get_tunnels()
print(tunnels)
if(tunnels):

  print(tunnels[0])
  for tunnel in tunnels:
      ngrok.disconnect(tunnel.public_url)
      print(f"Tunnel '{tunnel}' stopped successfully.")


ngrok_tunnel = ngrok.connect(8000)


import pyodbc
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

driver = "{ODBC Driver 18 for SQL Server}"
server = "tcp:ginieaiserver.database.windows.net,1433"
database = "GinieAIStoreDB"
username = "SQL"
password = "Anshul2411"

conn_str = f"DRIVER={driver};SERVER={server};DATABASE={database};Uid=SQL;Pwd={password};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"
print(pyodbc.drivers())
conn =  pyodbc.connect(conn_str)

engine = create_engine("mssql+pyodbc://", creator=lambda: conn)

SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)


def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()


db = next(get_db())
from sqlalchemy import text
update_stmt = text(f"UPDATE TOP(1) [dbo].[GinieEndpoints] SET AmazonProductService_Endpoint = '{ngrok_tunnel.public_url}';")

db.execute(update_stmt)
db.commit()

print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

[]
['ODBC Driver 18 for SQL Server']
Public URL: https://d0c7-34-16-176-90.ngrok-free.app


INFO:     Started server process [730]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     106.205.158.90:0 - "OPTIONS /analyze_review HTTP/1.1" 200 OK
https://www.amazon.in/dp/B08QP41KBP/ref=sspa_dk_detail_3?psc=1&pd_rd_i=B08QP41KBP&sp_csd=d2lkZ2V0TmFtZT1zcF9kZXRhaWxfdGhlbWF0aWM
90.0%
0.9
INFO:     106.205.158.90:0 - "POST /analyze_review HTTP/1.1" 200 OK
INFO:     106.205.158.90:0 - "POST / HTTP/1.1" 404 Not Found
INFO:     106.205.158.90:0 - "POST /analyze_review HTTP/1.1" 422 Unprocessable Entity
INFO:     106.205.158.90:0 - "POST /analyze_review HTTP/1.1" 422 Unprocessable Entity
INFO:     106.205.158.90:0 - "POST /analyze_review HTTP/1.1" 422 Unprocessable Entity
INFO:     106.205.158.90:0 - "POST /analyze_review HTTP/1.1" 422 Unprocessable Entity
INFO:     106.205.158.90:0 - "POST /analyze_review HTTP/1.1" 422 Unprocessable Entity
"https://www.amazon.in/dp/B08QP41KBP/ref=sspa_dk_detail_3?psc=1&pd_rd_i=B08QP41KBP&sp_csd=d2lkZ2V0TmFtZT1zcF9kZXRhaWxfdGhlbWF0aWM"
INFO:     106.205.158.90:0 - "POST /analyze_review HTTP/1.1" 200 OK
"https://www.amazon.in/dp/B08QP4

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-5bbb380dd855>", line 106, in <cell line: 106>
    uvicorn.run(app, port=8000)
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 575, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File 

90.0%
0.9
INFO:     106.205.158.90:0 - "POST /analyze_review HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [730]


KeyboardInterrupt: 